In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16, DenseNet201, ResNet50, InceptionV3, InceptionResNetV2
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from sklearn.metrics import classification_report, accuracy_score

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from math import sin, cos, sqrt, atan2, radians

def getGreatCircleDistance(lat1,lon1,lat2,lon2):
    R = 6371.009
    
    lat1 = radians(abs(lat1))
    lon1 = radians(abs(lon1))
    lat2 = radians(abs(lat2))
    lon2 = radians(abs(lon2))
    
    delta_lon = lon2 - lon1
    delta_lat = lat2 - lat1

    a = sin(delta_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(delta_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [3]:
df = pd.read_csv('/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/ProcessedImages/total_data.csv') # change this path to the one you want
df = df[['filename', 'latitude', 'longitude', 'level3']].set_index('filename')
df.head()

,latitude,longitude,level3
filename,,,
104728_cam-np3_frame5.jpg,41.53283,-8.81532,A4.1
104728_cam-np3_frame9.jpg,41.53283,-8.81531,A4.1
104728_cam-np3_frame50.jpg,41.53277,-8.81526,A4.1
104728_cam-np3_frame51.jpg,41.53277,-8.81526,A4.1
104728_cam-np3_frame52.jpg,41.53277,-8.81526,A4.1


In [4]:
unknown = pd.read_csv('/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/20180501/ProcessedImages/unknown.csv').set_index('filename') # change this path to the one you want
unknown = unknown.join(df, how='inner')
unknown.head()


,densenet,resnet,vgg,latitude,longitude,level3
filename,,,,,,
105317_cam-np3_frame1118.jpg,A3.1-A4.1,A3.1-A4.1,A3.1-A4.1,41.53074,-8.81473,NaN
105317_cam-np3_frame1129.jpg,A3.1-A4.1,A3.1-A4.1,A3.1-A4.1,41.53077,-8.81482,NaN
105317_cam-np3_frame1136.jpg,A5.1,A5.1,A5.1,41.53095,-8.81494,NaN
105317_cam-np3_frame1342.jpg,A3.1-A4.1,A3.1-A4.1,A3.1-A4.1,41.53152,-8.81545,NaN
105317_cam-np3_frame1343.jpg,A5.1,A5.1,A5.1,41.53152,-8.81545,NaN
105317_cam-np3_frame1344.jpg,A5.1,A5.1,A5.1,41.53152,-8.81545,NaN
105317_cam-np3_frame1345.jpg,A5.1,A5.1,A5.1,41.53152,-8.81545,NaN
105317_cam-np3_frame1346.jpg,A3.7-A4.7,A3.7-A4.7,A3.7-A4.7,41.53152,-8.81545,NaN
105317_cam-np3_frame1348.jpg,A5.1,A5.1,A5.1,41.53153,-8.81545,NaN


In [5]:
df_dense = unknown
df_res = unknown
df_vgg = unknown

In [6]:
df_dense.level3 = df_dense.densenet
df_res.level3 = df_dense.resnet
df_vgg.level3 = df_dense.vgg

df_dense = df_dense[['latitude', 'longitude', 'level3']]
df_res = df_dense[['latitude', 'longitude', 'level3']]
df_vgg = df_dense[['latitude', 'longitude', 'level3']]

In [7]:
df_dense = pd.concat([df_dense,df[['latitude', 'longitude', 'level3']].drop(df_dense.index)]).dropna()
df_res = pd.concat([df_res,df[['latitude', 'longitude', 'level3']].drop(df_dense.index)]).dropna()
df_vgg = pd.concat([df_vgg,df[['latitude', 'longitude', 'level3']].drop(df_dense.index)]).dropna()

In [9]:
sss_df = pd.read_csv('/media/diegues/be7ed47e-fd91-44a9-b591-4737c9a6fa3f/processed_sss/df.csv').set_index('filename') # change this path to the one you want
sss_df = sss_df[sss_df.date == '01/05/2018']
sss_df.head()
len(sss_df)

8312

In [ ]:
imagedata = []
for index, row in df_dense.replace(['A3.1', 'A4.1'],'A3.1-A4.1').replace(['A3.7', 'A4.7'],'A3.7-A4.7').iterrows():
    for index2, row2 in sss_df.iterrows():
        distance = getGreatCircleDistance(row['latitude'], row['longitude'], row2['latitude'], row2['longitude'])        
        if distance <= 0.005:
            if row2['auv'] == 'lauv-noptilus-3':
                if(distance <= 0.0035):
                    print(index,index2,distance)
                    imagedata.append(index2 + ',' + row['level3'] + ',' + str(distance))
            else:
                print(index,index2,distance)
                imagedata.append(index2 + ',' + row['level3'] + ',' + str(distance))

In [12]:
lcl_df = pd.DataFrame(columns=['sss_name', 'level3', 'distance'])
i = 0
for row in imagedata:
    lcl_df.loc[i] = row.split(',')
    i += 1


In [23]:
lcl_df.head()

,level3,distance
sss_name,,
105317_cam-np3-9-20,A3.1-A4.1,0.003251
105317_cam-np3-9-27,A3.1-A4.1,0.003123
105317_cam-np3-9-28,A3.1-A4.1,0.001458
105317_cam-np3-11-19,A3.1-A4.1,0.002032
105317_cam-np3-11-19,A5.1,0.002032


In [20]:
#lcl_df.to_csv('sss_data.csv', index=False)
lcl_df = pd.read_csv('sss_data.csv')

In [24]:
#lcl_df = lcl_df.set_index('sss_name')
new_levels = pd.get_dummies(lcl_df.level3)
#new_levels = lcl_df.drop(['level3', 'distance'], axis = 1).groupby('sss_name').sum()
new_levels.head()

,A3.1-A4.1,A3.7-A4.7,A5.1,A5.2,A5.4
sss_name,,,,,
105317_cam-np3-9-20,1,0,0,0,0
105317_cam-np3-9-27,1,0,0,0,0
105317_cam-np3-9-28,1,0,0,0,0
105317_cam-np3-11-19,1,0,0,0,0
105317_cam-np3-11-19,0,0,1,0,0
105317_cam-np3-11-19,0,0,1,0,0
105317_cam-np3-11-19,0,0,1,0,0
105317_cam-np3-11-19,0,1,0,0,0
105317_cam-np3-11-19,0,0,1,0,0


In [27]:

new_levels['most_frequent'] = new_levels[['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']].idxmax(axis=1)
lcl_df = pd.concat([lcl_df, new_levels], axis=1)

In [33]:
lcl_df = lcl_df.groupby('sss_name').sum()

In [35]:
lcl_df['y'] = lcl_df.idxmax(axis=1)

In [36]:
lcl_df.head()

,distance,A3.1-A4.1,A3.7-A4.7,A5.1,A5.2,A5.4,y
sss_name,,,,,,,
104728_cam-np3-1-34,0.050977,3,29,0,0,0,A3.7-A4.7
104728_cam-np3-1-43,0.006578,2,0,0,0,0,A3.1-A4.1
104728_cam-np3-1-44,0.002910,1,0,0,0,0,A3.1-A4.1
104728_cam-np3-2-3,0.002286,2,0,0,0,0,A3.1-A4.1
104728_cam-np3-2-4,0.002286,2,0,0,0,0,A3.1-A4.1
105317_cam-np3-1-17,0.051496,0,21,0,0,0,A3.7-A4.7
105317_cam-np3-1-35,0.042815,15,0,0,0,0,A3.1-A4.1
105317_cam-np3-1-36,0.070334,26,0,0,0,0,A3.1-A4.1
105317_cam-np3-10-19,0.072772,25,0,0,0,0,A3.1-A4.1


In [37]:
lcl_df.y.value_counts()

A3.1-A4.1    498
A5.1          27
A3.7-A4.7     22
A5.2          12
A5.4           4
Name: y, dtype: int64